In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = 'cpu'
import time, os, sys, pickle
import numpy as np
from scripts.utils import set_seed
from scripts.ds_class import *
from scripts.homeos import *
from scripts.plotting import *
from scripts.fit_motif import *
from scripts.time_series import *
from scripts.ra import *
from scripts.exp_tools import *

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.ticker import MaxNLocator
import seaborn as sns

set_seed(313)
from pathlib import Path
exp_dir = Path('experiments')
data_dir = exp_dir / 'all_targets'

In [11]:
npy_files = list(data_dir.glob('*.npy'))
print(data_dir)
for file in npy_files:
    target_name = file.name.removesuffix('.npy')

    target_trajs = np.load(file)
    print(target_name, 'dim = ', target_trajs.shape[2])
    file_ = data_dir / f'{target_name}.npy'

experiments\all_targets
avi dim =  64
bila_trajs dim =  2
homeo_pert_ring dim =  2
lienard_sigmoid dim =  2
noisy_vdp dim =  2
ra dim =  2
ra_nstd0.05 dim =  2
ra_nstd0.1 dim =  2
ra_nstd0.25 dim =  2
selkov dim =  2
vdp dim =  2
vf_pert_ring dim =  2


# 0D: LDS
# 1D: Bounded line attractor (bla)
# 2D archetype: 
- Ring attractor
- Limit cycle
- Bistable


# 2D targets:
- Van der Pol 
- noisy VdP
- Lienard
- Selkov
- RA 
- noisy RA
- deformed RA
- perturbed RA
- 2 Bounded Line Attractors (2 BLAs)

In [6]:
archetypes_2d = ['lds', 'lc', 'ring', 'bla', 'bistable']

In [7]:
# save_dir = data_dir / 'motif_fits' 
# save_dir.mkdir(parents=True, exist_ok=True)

# for file in npy_files:
#     target_name = file.name.removesuffix('.npy')
#     print("Starting training archetype on system", target_name)
#     for archetype in archetypes_2d:
#         print('training archetype', archetype)
#         run_on_target(target_name, save_dir=save_dir, data_dir=data_dir, ds_motif=archetype, analytic=True, canonical=True, jac_lambda_reg=.0)

In [8]:
def format_reg_name(reg: float) -> str:
    if reg == 0:
        return "0"
    elif reg >= 0.1:
        return f"{reg:.1f}"       # e.g., 1.0 → '1.0', 0.1 → '0.1'
    else:
        return f"{reg:.0e}"       # e.g., 0.001 → '1e-3'

In [12]:
reg_values = [1., 0.1, 0.01, 0.001, 0.0001, 0.]
archetypes_2d = ['ring']
file = npy_files[1]

target_name = file.name.removesuffix('.npy')
file

WindowsPath('experiments/all_targets/bila_trajs.npy')

In [ ]:
for reg_value in reg_values:
    name = f"seqjacreg_idinit_{format_reg_name(reg_value)}"
    save_dir = data_dir / "motif_fits" / name
    save_dir.mkdir(parents=True, exist_ok=True)
    archetype ='ring'
    if reg_value == 1:
        load_hdsnet_path = None
    else:
        load_hdsnet_path = save_dir_prev / archetype / f'homeo_{target_name}.pth'
    print('training archetype', archetype, 'with reg', reg_value)
    run_on_target(target_name, save_dir=save_dir, data_dir=data_dir, ds_motif=archetype, lr=1e-3,
                    analytic=True, canonical=True, num_epochs=100, two_phase=True, second_phase_epochs=100,
                        jac_lambda_reg=reg_value, load_hdsnet_path=load_hdsnet_path,
                        homeo_init_type = "small", homeo_init_std=1e-4)
    save_dir_prev = save_dir

In [ ]:
# # noisy vdp exp
# noisy_vdp_folder = 'experiments/lcs/noisy_vdp'
# data_dir = exp_dir / 'lcs' / 'noisy_vdp'

# save_dir.mkdir(parents=True, exist_ok=True)
# npy_files = list(data_dir.glob('*.npy'))
# print(data_dir)
# for file in npy_files:
#     target_name = file.name.removesuffix('.npy')

#     target_trajs = np.load(file)
#     print(target_name, 'dim = ', target_trajs.shape[2])
#     file_ = data_dir / f'{target_name}.npy'

# file = npy_files[0]
# target_name = file.name.removesuffix('.npy')
# print("Starting training motifs on system", target_name)
# for archetype in archetypes_2d:
#     print('training archetype', archetype)
#     run_on_target(target_name, save_dir=save_dir, data_dir=data_dir, ds_motif=archetype, analytic=True, canonical=True)

In [ ]:
# #bla pert
# from pathlib import Path
# exp_dir = Path('experiments')

# data_dir = exp_dir / 'bla_pert' 
# save_dir = data_dir / 'motif_fits'
# save_dir.mkdir(parents=True, exist_ok=True)
# npy_files = list(data_dir.glob('*.npy'))
# print(data_dir)

# for file in npy_files:
#     target_name = file.name.removesuffix('.npy')

#     target_trajs = np.load(file)
#     print(target_name)
#     file_ = data_dir / f'{target_name}.npy'
#     for archetype in archetypes_2d:
#         print('training archetype', archetype)
#         run_on_target(target_name, save_dir=save_dir, data_dir=data_dir, ds_motif=archetype, analytic=True, canonical=True)